# [In Progress]

# Programación Entera: Heurísticas para Problemas de Redes

## Descripción

[TODO]

## Autor
  
  * Sergio García Prado - [garciparedes.me](https://garciparedes.me)
  
## Fecha

  * Mayo de 2018


## Contenidos
  
  * [Introducción](#Introducción)
  * [Problemas De Redes](#Problemas-de-Redes)
    * [Flujo de Redes con Costes Fijos](#Flujo-de-Redes-con-Costes-Fijos)
    * [Árbol Recubridor Mínimo](#Árbol-Recubridor-Mínimo)
    * [Árbol Recubridor Mínimo Capacitado](#Árbol-Recubridor-Mínimo-Capacitado)
    * [Problema de Steiner](#Problema-de-Steiner)
  * [Conclusiones](#Conclusiones)

## Introducción

TODO

In [1]:
rm(list = ls())

In [2]:
library(slam, warn.conflicts = FALSE)
library(Rglpk, warn.conflicts = FALSE)
library(purrr, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)
library(magrittr, warn.conflicts = FALSE)
library(ggplot2, warn.conflicts = FALSE)
library(ggthemes, warn.conflicts = FALSE)
library(scales, warn.conflicts = FALSE)
library(igraph, warn.conflicts = FALSE)

Using the GLPK callable library version 4.65


In [3]:
options(repr.matrix.max.rows = 600, repr.matrix.max.cols = 600)

#### Constantes y Funciones de Apoyo

In [4]:
dataset.root <- "./data/"
dataset.folder <- paste0(dataset.root, "network/")

In [5]:
simple.file.names <- c("gr15_1.dat", "gr15_2.dat", "gr15_3.dat", "gr15_4.dat", "gr15_4.dat" )

In [6]:
Solve <- function(...) {
    Rglpk_solve_LP(...)
}

In [7]:
ReadRaw <- function(file.path) {
    con <- 
        file.path %>%
        file(open = "r")
    raw <- 
        con %>%
        readLines(warn = FALSE) %>% 
        strsplit("[[:blank:]]") %>%
        unlist() %>%
        as.double()
    close(con)
    return(raw)
}

## Problemas de Redes

TODO

### Flujo de Redes con Costes Fijos

#### Descripción

TODO

#### Modelo

TODO

\begin{equation}
    \begin{array}{ll@{}ll}
      \text{Minimizar} & \displaystyle \sum\limits_{(i,j) \in A}\left(f_{ij} \cdot y_{ij} + \sum_{k=1}^{p} c_{ijk} \cdot x_{ijk}\right) \\
      \text{sujeto a}  & \sum\limits_{j} x_{ijk} - \sum\limits_{j} x_{jik} = b_{ik}, & \forall i \in N, \forall k \in \{1,...,p\}\\
                       & x_{ijk} \leq u_{ijk} \cdot y_{j},  & \forall (i,j) \in A, \forall k \in \{1,...,p\} \\
                       & x_{ijk} \geq 0, & \forall (i,j) \in A, \forall k \in \{1,...,p\} \\
                       & y_{ij} \in \{0, 1\},  & \forall (i,j) \in A
    \end{array}
\end{equation}

#### Ejemplos

TODO

##### distribucion.gas

TODO(@garciparedes): Do Example.

##### aguas.residuales

TODO(@garciparedes): Do Example.

##### un.producto

TODO(@garciparedes): Do Example.

##### avicola

TODO(@garciparedes): Do Example.

##### mercasol

TODO(@garciparedes): Do Example.

### Árbol Recubridor Mínimo

#### Descripción

TODO

#### Modelo

TODO

#### Ejemplos

TODO

##### matriz.12x12

TODO(@garciparedes): Do Example.

In [8]:
matriz.12x12 <- 
    "matriz_12x12.dat" %>%
    { paste0(dataset.folder, .) } %>%
    ReadRaw() %>%
    matrix(ncol = 12)

##### berlin.52

TODO(@garciparedes): Do Example.

##### st.70

TODO(@garciparedes): Do Example.

##### a.280

TODO(@garciparedes): Do Example.

##### gr15.{1,...,5}

TODO(@garciparedes): Do Example.

### Árbol Recubridor Mínimo Capacitado

#### Descripción

TODO

#### Modelo

TODO

#### Ejemplos

TODO

##### gr15.{1,...,5}

TODO(@garciparedes): Do Example.

### Problema de Steiner

#### Descripción

TODO

#### Modelo

TODO

#### Ejemplos

TODO

##### grafo.11.nodos

TODO(@garciparedes): Do Example.

##### gr15.{1,...,5}

TODO(@garciparedes): Do Example.

## Conclusiones

TODO

## Referencias

  * [TRC13] Team, R.C., 2013. R: A language and environment for statistical computing.
  * [GP18] Sergio García Prado. Programación Entera: Heurísticas, 2018. [github.com/garciparedes/integer-programming-heuristics](https://github.com/garciparedes/integer-programming-heuristics).
  * [SA18] Jesús Sáez Aguado. Programación Entera, 2017/18. Facultad de Ciencias: Departamento de Estadística e Investigación Operativa.
  * [THBSST17] Theussl, S., Hornik, K., Buchta, C., Schwendinger, F., Schuchardt, H. and Theussl, M.S., 2017. Package ‘Rglpk’.
  